In [ ]:
import pydicom as dcm
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import skimage
from skimage import measure

In [ ]:
## explore metadata
mdata = pd.read_csv("metadata.csv")
mdata

In [ ]:
len(np.unique(mdata["Patient ID"])), len(np.unique(mdata["Study Instance UID"])), len(np.unique(mdata["Series Instance UID"]))

In [ ]:
unique_series_uids = list(np.unique(mdata["Series Instance UID"]))

In [ ]:
duplicate_series_uids = list()
for i in unique_series_uids:
    if np.count_nonzero(np.asarray(mdata["Series Instance UID"]) == i) > 2:
        duplicate_series_uids.append(i)

In [ ]:
duplicate_mdata = mdata[mdata["Series Instance UID"].isin(duplicate_series_uids)]
duplicate_mdata

In [ ]:
test_mdata = mdata[mdata["Series Instance UID"]==duplicate_series_uids[8]]
test_mdata

In [ ]:
test_dcm_ids = list(test_mdata["SOP Instance UID"])
test_dcm_ids

In [ ]:
## transfer testing dicom images to local
for i in test_dcm_ids:
    os.system(f"gsutil -m cp gs://rsna_hemorrhage/dataset/stage_2_train/{i}.dcm .")

In [ ]:
def window_image(dcm, window_key, window_dict):
    window_center, window_width = window_dict[window_key]
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        print("Bit Standardization")
        dcm_bit_standardize(dcm)

    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

In [ ]:
window_dict = {
    "brain": [40, 80],
    "subdural": [80, 200],
    "soft_tissue": [40, 380]
}

In [ ]:
scans = [dcm.dcmread(f"{i}.dcm") for i in test_dcm_ids]

In [ ]:
test_dcm_imgs = [dcm.dcmread(f"{i}.dcm").pixel_array for i in test_dcm_ids]
# test_dcm_imgs = [window_image(i, "brain", window_dict) for i in scans]
# test_dcm_imgs = [window_image(i, "subdural", window_dict) for i in scans]
# test_dcm_imgs = [window_image(i, "soft_tissue", window_dict) for i in scans]

In [ ]:
reconstruct_3d = np.stack(test_dcm_imgs[z] for z in range(len(test_dcm_ids)))
reconstruct_3d = reconstruct_3d.astype(np.int16)
reconstruct_3d[reconstruct_3d == -2000] = 0
intercept = scans[0].RescaleIntercept
slope = scans[0].RescaleSlope
if slope != 1:
    reconstruct_3d = slope * reconstruct_3d.astype(np.float64)
    reconstruct_3d = reconstruct_3d.astype(np.int16)

reconstruct_3d += np.int16(intercept)
reconstruct_3d = np.array(reconstruct_3d, dtype=np.int16)

In [ ]:
for i in range(len(test_dcm_ids)):
    plt.imshow(test_dcm_imgs[i], cmap=plt.cm.gray)
    plt.savefig(f"data/{i}.png")

In [ ]:
plt.imshow(reconstruct_3d.transpose(2,1,0), cmap=plt.cm.gray)

In [ ]:
def plot_3d(image, threshold=-300): 
    p = image.transpose(2,1,0)
    verts, faces, normals, values = measure.marching_cubes(p, threshold)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()

In [ ]:
plot_3d(image=reconstruct_3d, threshold=-300)

In [ ]:
import cv2
import os

image_folder = 'data'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))

height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()